In [1]:
import numpy as np
import sys
from os.path import join

dir_path = "/Users/jeremygoldwasser/Desktop/RankSHAP"
data_path = join(dir_path, "Experiments", "Data")
sys.path.append(join(dir_path, "HelperFiles"))
from helper2 import *
from shapley_sampling3 import *
from train_models2 import *
from load_data import *

import warnings
warnings.filterwarnings('ignore')



/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load dataset and train model

In [5]:
dataset = "census"
# X_train, y_train, X_test, y_test, mapping_dict = make_data(data_path, dataset)
# save_data(data_path, dataset, X_train, y_train, X_test, y_test, mapping_dict)
X_train, y_train, X_test, y_test, mapping_dict = load_data(data_path, dataset)
xloc = X_test[1:2]

In [6]:
model = train_neural_net(X_train, y_train)

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))

Class imbalance: 75%
NN 82% accuracy


## Preliminary run of Shapley sampling

Not infrequently, expected number of samples to be significant exceeds max number of perms; sampling it, though, it's significant. This suggests our distribution is too conservative. Maybe we can remove the factor of 2.

In [10]:
# %run shapley_sampling2
np.random.seed(6)

shap_vals, diffs_all_feats = rankshap(model, X_train, xloc, K=2, alpha=0.2, mapping_dict=mapping_dict, max_n_perms=10000, abs=True)
get_ordering(shap_vals, abs=True)

In [8]:
print(np.sum(shap_vals), model(xloc) - np.mean(model(X_train)))

print(np.sort(np.abs(shap_vals))[::-1])

0.524535798325114 0.53665346
[0.28216124 0.10702227 0.07026205 0.0640175  0.04771363 0.0221793
 0.0179747  0.01356653 0.01060656 0.00572422 0.00252114 0.00032385]


# Simulate Experimental FWER

In [15]:
# %run shapley_sampling2
# np.random.seed(2) # Unusual
K = 5
top_K = []
for i in range(5):
    # print(i)
    shap_vals2, diffs_all_feats2 = rankshap(model, X_train, xloc, K=K, alpha=0.2, 
                            mapping_dict=mapping_dict, abs=True, n_equal=False)
    if shap_vals2 != "NA":
        est_top_K = get_ordering(shap_vals2)[:K]
        top_K.append(est_top_K)
        print(len(top_K))
        # if (i+1) % 5==0: 
        #     print(i+1)

1
2
3
4
5


In [20]:
print(calc_fwer(top_K))
print([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))])

0.0
[4184, 100, 2570, 100, 7131, 100, 100, 2044, 792, 100, 100, 2083]
